In [ ]:
#RGB画像の他人受入率の評価プログラム
import keras 
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten
from keras.layers import Conv2D,MaxPooling2D
from keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image


def def_model(in_shape, nb_classes):
    model = Sequential()
    model.add(Conv2D(32,
              kernel_size=(3, 3),
              activation='relu',
              input_shape=in_shape))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes, activation='softmax'))
    return model


def get_model(in_shape, nb_classes):
    model = def_model(in_shape, nb_classes)
    model.compile(
        loss='categorical_crossentropy',
        optimizer=RMSprop(),
        metrics=['accuracy'])
    return model

#12-25修正
#他人受入率用のテスト配列
omote=[#"gotou/omote",
#         "isii/omote",
#         "mimura/omote",
#         "nakamura/omote",
#         "okutani/omote",
        "saitouhide/omote",
        "saitouryou/omote",
        "sakuma/omote",
        "tanaka/omote"]
        #"tochikubo/omote"]
ura=[#"gotou/ura",
#      "isii/ura",
#      "mimura/ura",
#      "nakamura/ura",
#      "okutani/ura",
     "saitouhide/ura",
     "saitouryou/ura",
     "sakuma/ura",
     "tanaka/ura"]
     #"tochikubo/ura"]

target_omote=[]
target_ura=[]
passpercent=[]
notpasspercent=[]
omotepercent=[]
urapercent=[]

#12-25修正
for k in range(4):#変更必要
    for i in range(10):
        target_omote.append("../test/"+str(omote[k])+"/"+str(i)+".jpg")#変更必要
        target_ura.append("../test/"+str(ura[k])+"/"+str(i)+".jpg")#変更必要

im_rows = 256 #変更必要
im_cols = 256 #変更必要
im_color = 3 #変更必要
in_shape = (im_rows, im_cols, im_color)
nb_classes = 40 #変更必要
#配列の最大値　11月追加
max_score = 0.0

#12-25修正
#ラベル
LABELS=["相澤さんの表","相澤さんの裏",
       "新井さんの表","新井さんの裏",
       "江波戸の表","江波戸の裏",
       "長谷川さんの表","長谷川さんの裏",
       "樋口さんの表","樋口さんの裏",
       "石井さんの表","石井さんの裏",
       "小島さんの表","小島さんの裏",
       "中村さんの表","中村さんの裏",
       "大野さんの表","大野さんの裏",
       "坂井さんの表","坂井さんの裏",
       "関根さんの表","関根さんの裏",
       "島田さんの表","島田さんの裏",
       "清水さんの表","清水さんの裏",
       "高橋さんの表","高橋さんの裏",
       "田中ひろきさんの表","田中ひろきさんの裏",
       "田中みゆさんの表","田中みゆさんの裏",
       "田中たいきさんの表","田中たいきさんの裏",
       "徳山さんの表","徳山さんの裏",
       "渡邉さんの表","渡邉さんの裏",
       "山口さんの表","山口さんの裏"]

model = get_model(in_shape, nb_classes)
model.load_weights('./2023hdf/2023num40RGB256pixbatch256.hdf5')#変更必要

#最大値を見つける関数 11月追加
def findmaxscore(per):
    global max_score
    if per!=100.0:
        if max_score<=per:
            max_score=per 
            
def check_photo(path):
    img=Image.open(path)
    img=img.resize((im_cols,im_rows))
    plt.imshow(img)
    plt.show()

    x=np.asarray(img)
    x=x.reshape(-1,im_rows,im_cols,im_color)
    x=x/255
    
    #予測　
    pre=model.predict([x])[0]#画像の予測
    idx=pre.argmax()
    per=(pre[idx]*100)
    if per<99.99999 and ("omote" in path): #閾値
        omotepercent.append(per)
    elif per<99.99999 and ("ura" in path): #閾値
        urapercent.append(per)
    else:
        pass
    return(idx,per)

def check_photo_str(path):
    idx,per=check_photo(path)
    if per>=99.99999: #閾値
        print("この写真は、",LABELS[idx],"です。")
        print("可能性は、",per,"%です。")
        passpercent.append(per)
        #配列の最大値を見つける　11月追加
        findmaxscore(per)
    else:
        print('\033[31m'+"可能性が",per,"%なので認証できません。"+'\033[31m')
        print("認証の結果は",LABELS[idx],"でした。")
        notpasspercent.append(per)
    
if __name__=='__main__':
    number=[9,19,29,39,49,59,69,79,89,99,109,119,129,139,149]
    for i in range(40):
        print("\nここからは"+str(i)+"番目の写真です")
        check_photo_str(target_omote[i])
        for l in range(15):
            if i==number[l]:
                print("================================================")
                print("テスト対象の人が変わります")
                print("================================================")
    for i in range(40):
        print("\nここからは"+str(i)+"番目の写真です")
        check_photo_str(target_ura[i])
        for l in range(15):
            if i==number[l]:
                print("================================================")
                print("テスト対象の人が変わります")
                print("================================================")
   
    print(passpercent)
    print(notpasspercent)
    print("他人受入に用いる個数"+str(len(passpercent)))
    FAR=len(passpercent)
    print("閾値未満で識別したテスト数"+str(len(notpasspercent)))
    print("表の手だけで閾値未満で識別したテスト数"+str(len(omotepercent)))
    print("裏の手だけで閾値未満で識別したテスト数"+str(len(urapercent)))
    totalpass=len(passpercent)/80 #12-26修正
    totalnotpass=len(notpasspercent)/80
    omotenotpass=len(omotepercent)/40
    omotepass=(40-len(omotepercent))/40
    uranotpass=len(urapercent)/40
    urapass=(40-len(urapercent))/40
    print("================================================")
    #未学習の手を閾値未満で識別した割合
    print("他人拒否率"+str(round(totalnotpass*100))+"%")
    #未学習の手を閾値以上で識別した割合
    print("他人受入率"+str(round(totalpass*100))+"%")
    print("表の手だけで未学習の手を閾値未満で識別した割合"+str(round(omotenotpass*100))+"%")
    print("表の手だけで未学習の手を閾値以上で識別した割合"+str(round(omotepass*100))+"%")
    print("裏の手だけで未学習の手を閾値未満で識別した割合"+str(round(uranotpass*100))+"%")
    print("裏の手だけで未学習の手を閾値以上で識別した割合"+str(round(urapass*100))+"%")
    print("閾値以上の最大値は"+str(max_score)) #11月追加
    print("閾値以上の最小値は"+str(min(passpercent))) #11月追加

In [ ]:
#グレイスケール画像の他人受入率の評価プログラム
import keras 
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten
from keras.layers import Conv2D,MaxPooling2D
from keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image


def def_model(in_shape, nb_classes):
    model = Sequential()
    model.add(Conv2D(32,
              kernel_size=(3, 3),
              activation='relu',
              input_shape=in_shape))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes, activation='softmax'))
    return model


def get_model(in_shape, nb_classes):
    model = def_model(in_shape, nb_classes)
    model.compile(
        loss='categorical_crossentropy',
        optimizer=RMSprop(),
        metrics=['accuracy'])
    return model

#他人受入率用のテスト配列
omote=[#"gotou/omote",
#         "isii/omote",
#         "mimura/omote",
#        "nakamura/omote",
#         "okutani/omote",
        "saitouhide/omote",
        "saitouryou/omote",
       "sakuma/omote",
       "tanaka/omote"]
       #"tochikubo/omote"]
ura=[#"gotou/ura",
#      "isii/ura",
#      "mimura/ura",
#       "nakamura/ura",
#       "okutani/ura",
      "saitouhide/ura",
      "saitouryou/ura",
      "sakuma/ura",
      "tanaka/ura"]
     # "tochikubo/ura"]

target_omote=[]
target_ura=[]
passpercent=[]
notpasspercent=[]
omotepercent=[]
urapercent=[]

#12-25修正
for k in range(4):#変更必要
    for i in range(10):
        target_omote.append("../graytest/"+str(omote[k])+"/"+str(i)+".jpg")#変更必要
        target_ura.append("../graytest/"+str(ura[k])+"/"+str(i)+".jpg")#変更必要

im_rows = 256 #変更必要
im_cols = 256 #変更必要
im_color = 1 #変更必要
in_shape = (im_rows, im_cols, im_color)
nb_classes = 40 #変更必要
#配列の最大値　11月追加
max_score = 0.0

#12-25修正
#ラベル
LABELS=["相澤さんの表","相澤さんの裏",
       "新井さんの表","新井さんの裏",
       "江波戸の表","江波戸の裏",
       "長谷川さんの表","長谷川さんの裏",
       "樋口さんの表","樋口さんの裏",
       "石井さんの表","石井さんの裏",
       "小島さんの表","小島さんの裏",
       "中村さんの表","中村さんの裏",
       "大野さんの表","大野さんの裏",
       "坂井さんの表","坂井さんの裏",
       "関根さんの表","関根さんの裏",
       "島田さんの表","島田さんの裏",
       "清水さんの表","清水さんの裏",
       "高橋さんの表","高橋さんの裏",
       "田中ひろきさんの表","田中ひろきさんの裏",
       "田中みゆさんの表","田中みゆさんの裏",
       "田中たいきさんの表","田中たいきさんの裏",
       "徳山さんの表","徳山さんの裏",
       "渡邉さんの表","渡邉さんの裏",
       "山口さんの表","山口さんの裏"]


model = get_model(in_shape, nb_classes)
model.load_weights('./2023hdf/2023num40gray256pixbatch256.hdf5')#変更必要

#最大値を見つける関数 11月追加
def findmaxscore(per):
    global max_score
    if per!=100.0:
        if max_score<=per:
            max_score=per 

def check_photo(path):
    img=Image.open(path)
    img=img.resize((im_cols,im_rows))
    plt.imshow(img,cmap='gray')
    plt.show()

    x=np.asarray(img)
    x=x.reshape(-1,im_rows,im_cols,im_color)
    x=x/255
    
    #予測
    pre=model.predict([x])[0]#画像の予測
    idx=pre.argmax()
    per=(pre[idx]*100)
    if per<99.99999 and ("omote" in path): #閾値変更箇所
        omotepercent.append(per)
    elif per<99.99999 and ("ura" in path): #閾値変更箇所
        urapercent.append(per)
    else:
        pass
    return(idx,per)

def check_photo_str(path):
    idx,per=check_photo(path)
    if per>=99.99999: #閾値変更箇所
        print("この写真は、",LABELS[idx],"です。")
        print("可能性は、",per,"%です。")
        passpercent.append(per)
        #配列の最大値を見つける　11月追加
        findmaxscore(per)
    else:
        print('\033[31m'+"可能性が",per,"%なので認証できません。"+'\033[31m')
        print("認証の結果は",LABELS[idx],"でした。")
        notpasspercent.append(per)
    
if __name__=='__main__':
    number=[9,19,29,39,49,59,69,79,89,99,109,119,129,139,149]
    for i in range(40):
        print("\nここからは"+str(i)+"番目の写真です")
        check_photo_str(target_omote[i])
        for l in range(15):
            if i==number[l]:
                print("================================================")
                print("テスト対象の人が変わります")
                print("================================================")
    for i in range(40):
        print("\nここからは"+str(i)+"番目の写真です")
        check_photo_str(target_ura[i])
        for l in range(15):
            if i==number[l]:
                print("================================================")
                print("テスト対象の人が変わります")
                print("================================================")
   
    print(passpercent)
    print(notpasspercent)
    print("他人受入に用いる個数"+str(len(passpercent)))
    FAR=len(passpercent)
    print("閾値未満で識別したテスト数"+str(len(notpasspercent)))
    print("表の手だけで閾値未満で識別したテスト数"+str(len(omotepercent)))
    print("裏の手だけで閾値未満で識別したテスト数"+str(len(urapercent)))
    totalpass=len(passpercent)/80 #12-26修正
    totalnotpass=len(notpasspercent)/80
    omotenotpass=len(omotepercent)/40
    omotepass=(40-len(omotepercent))/40
    uranotpass=len(urapercent)/40
    urapass=(40-len(urapercent))/40
    print("================================================")
    #未学習の手を閾値未満で識別した割合
    print("他人拒否率"+str(round(totalnotpass*100))+"%")
    #未学習の手を閾値以上で識別した割合
    print("他人受入率"+str(round(totalpass*100))+"%")
    print("表の手だけで未学習の手を閾値未満で識別した割合"+str(round(omotenotpass*100))+"%")
    print("表の手だけで未学習の手を閾値以上で識別した割合"+str(round(omotepass*100))+"%")
    print("裏の手だけで未学習の手を閾値未満で識別した割合"+str(round(uranotpass*100))+"%")
    print("裏の手だけで未学習の手を閾値以上で識別した割合"+str(round(urapass*100))+"%")
    print("閾値以上の最大値は"+str(max_score)) #11月追加
    print("閾値以上の最小値は"+str(min(passpercent))) #11月追加

In [ ]:
#エッジ画像カーネル3の他人受入率の評価プログラム
import keras 
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten
from keras.layers import Conv2D,MaxPooling2D
from keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import cv2
from PIL import Image


def def_model(in_shape, nb_classes):
    model = Sequential()
    model.add(Conv2D(32,
              kernel_size=(3, 3),
              activation='relu',
              input_shape=in_shape))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes, activation='softmax'))
    return model


def get_model(in_shape, nb_classes):
    model = def_model(in_shape, nb_classes)
    model.compile(
        loss='categorical_crossentropy',
        optimizer=RMSprop(),
        metrics=['accuracy'])
    return model

#他人受入率用のテスト配列
omote=[#"gotou/omote",
#         "isii/omote",
#         "mimura/omote",
#        "nakamura/omote",
#         "okutani/omote",
        "saitouhide/omote",
        "saitouryou/omote",
        "sakuma/omote",
        "tanaka/omote"]
       # "tochikubo/omote"]
ura=[#"gotou/ura",
#      "isii/ura",
#      "mimura/ura",
#       "nakamura/ura",
#       "okutani/ura",
      "saitouhide/ura",
      "saitouryou/ura",
      "sakuma/ura",
      "tanaka/ura"]
      #"tochikubo/ura"]

target_omote=[]
target_ura=[]
laplace3=[[1,1,1],
         [1,-8,1],
         [1,1,1]]
passpercent=[]
notpasspercent=[]
omotepercent=[]
urapercent=[]


#12-25修正
for k in range(4):#変更必要
    for i in range(10):
        target_omote.append("../graytest/"+str(omote[k])+"/"+str(i)+".jpg")#変更必要
        target_ura.append("../graytest/"+str(ura[k])+"/"+str(i)+".jpg")#変更必要


im_rows = 256 #変更必要
im_cols = 256 #変更必要
im_color = 1 #変更必要
in_shape = (im_rows, im_cols, im_color)
nb_classes = 40 #変更必要
#配列の最大値　11月追加
max_score = 0.0

#12-25修正
#ラベル
LABELS=["相澤さんの表","相澤さんの裏",
       "新井さんの表","新井さんの裏",
       "江波戸の表","江波戸の裏",
       "長谷川さんの表","長谷川さんの裏",
       "樋口さんの表","樋口さんの裏",
       "石井さんの表","石井さんの裏",
       "小島さんの表","小島さんの裏",
       "中村さんの表","中村さんの裏",
       "大野さんの表","大野さんの裏",
       "坂井さんの表","坂井さんの裏",
       "関根さんの表","関根さんの裏",
       "島田さんの表","島田さんの裏",
       "清水さんの表","清水さんの裏",
       "高橋さんの表","高橋さんの裏",
       "田中ひろきさんの表","田中ひろきさんの裏",
       "田中みゆさんの表","田中みゆさんの裏",
       "田中たいきさんの表","田中たいきさんの裏",
       "徳山さんの表","徳山さんの裏",
       "渡邉さんの表","渡邉さんの裏",
       "山口さんの表","山口さんの裏"]

model = get_model(in_shape, nb_classes)
model.load_weights('./2023hdf/2023num40karnel3-256pxbatch256.hdf5')#変更必要

#最大値を見つける関数 11月追加
def findmaxscore(per):
    global max_score
    if per!=100.0:
        if max_score<=per:
            max_score=per 

def check_photo(path):
    img=cv2.imread(path)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    karnel=np.array(laplace3 , np.float64)#変更必要
    img=cv2.filter2D(img,-1,karnel)
    img=cv2.resize(img,(im_rows,im_cols))
    plt.imshow(img,cmap="gray")
    plt.show()

    x=np.asarray(img)
    x=x.reshape(-1,im_rows,im_cols,im_color)
    x=x/255
    
    #予測
    pre=model.predict([x])[0]#画像の予測
    idx=pre.argmax()
    per=(pre[idx]*100)
    #----------97.9---98.5---98.9---99.5---99.9---99,95---99.99-------------
    if per < 99.99999 and ("omote" in path): #閾値変更箇所
        omotepercent.append(per)
    elif per < 99.99999 and ("ura" in path): #閾値変更箇所
        urapercent.append(per)
    else:
        pass
    return(idx,per)

def check_photo_str(path):
    idx,per=check_photo(path)
    if per >= 99.99999: #閾値変更箇所
        print("この写真は、",LABELS[idx],"です。")
        print("可能性は、",per,"%です。")
        passpercent.append(per)
        #配列の最大値を見つける　11月追加
        findmaxscore(per)
    else:
        print('\033[31m'+"可能性が",per,"%なので認証できません。"+'\033[31m')
        print("認証の結果は",LABELS[idx],"でした。")
        notpasspercent.append(per)
    
if __name__=='__main__':
    number=[9,19,29,39,49,59,69,79,89,99,109,119,129,139,149]
    for i in range(40):
        print("\nここからは"+str(i)+"番目の写真です")
        check_photo_str(target_omote[i])
        for l in range(15):
            if i==number[l]:
                print("================================================")
                print("テスト対象の人が変わります")
                print("================================================")
    for i in range(40):
        print("\nここからは"+str(i)+"番目の写真です")
        check_photo_str(target_ura[i])
        for l in range(15):
            if i==number[l]:
                print("================================================")
                print("テスト対象の人が変わります")
                print("================================================")
                
    print(passpercent)
    print(notpasspercent)
    print("他人受入に用いる個数"+str(len(passpercent)))
    FAR=len(passpercent)
    print("閾値未満で識別したテスト数"+str(len(notpasspercent)))
    print("表の手だけで閾値未満で識別したテスト数"+str(len(omotepercent)))
    print("裏の手だけで閾値未満で識別したテスト数"+str(len(urapercent)))
    totalpass=len(passpercent)/80 #12-26修正
    totalnotpass=len(notpasspercent)/80
    omotenotpass=len(omotepercent)/40
    omotepass=(40-len(omotepercent))/40
    uranotpass=len(urapercent)/40
    urapass=(40-len(urapercent))/40
    print("================================================")
    #未学習の手を閾値未満で識別した割合
    print("他人拒否率"+str(round(totalnotpass*100))+"%")
    #未学習の手を閾値以上で識別した割合
    print("他人受入率"+str(round(totalpass*100))+"%")
    print("表の手だけで未学習の手を閾値未満で識別した割合"+str(round(omotenotpass*100))+"%")
    print("表の手だけで未学習の手を閾値以上で識別した割合"+str(round(omotepass*100))+"%")
    print("裏の手だけで未学習の手を閾値未満で識別した割合"+str(round(uranotpass*100))+"%")
    print("裏の手だけで未学習の手を閾値以上で識別した割合"+str(round(urapass*100))+"%")
    print("閾値以上の最大値は"+str(max_score)) #11月追加
    try:
        print("閾値以上の最小値は"+str(min(passpercent))) #11月追加 
    except:
        pass

In [ ]:
#エッジ画像カーネル5の他人受入率の評価プログラム
import keras 
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten
from keras.layers import Conv2D,MaxPooling2D
from keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import cv2
from PIL import Image


def def_model(in_shape, nb_classes):
    model = Sequential()
    model.add(Conv2D(32,
              kernel_size=(3, 3),
              activation='relu',
              input_shape=in_shape))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes, activation='softmax'))
    return model


def get_model(in_shape, nb_classes):
    model = def_model(in_shape, nb_classes)
    model.compile(
        loss='categorical_crossentropy',
        optimizer=RMSprop(),
        metrics=['accuracy'])
    return model

#他人受入率用のテスト配列
omote=["gotou/omote",
        "isii/omote",
        "mimura/omote",
       "nakamura/omote",
        "okutani/omote",
#         "saitouhide/omote",
#         "saitouryou/omote",
#        "sakuma/omote",
#        "tanaka/omote"]
       "tochikubo/omote"]
ura=["gotou/ura",
     "isii/ura",
     "mimura/ura",
      "nakamura/ura",
      "okutani/ura",
#       "saitouhide/ura",
#       "saitouryou/ura",
#       "sakuma/ura",
#       "tanaka/ura"]
      "tochikubo/ura"]

target_omote=[]
target_ura=[]
laplace5=[[-1,-3,-4,-3,-1],
         [-3,0,6,0,-3],
         [-4,6,20,6,-4],
         [-3,0,6,0,-3],
         [-1,-3,-4,-3,-1]]
passpercent=[]
notpasspercent=[]
omotepercent=[]
urapercent=[]


#12-25修正
for k in range(6):#変更必要
    for i in range(10):
        target_omote.append("../graytest/"+str(omote[k])+"/"+str(i)+".jpg")#変更必要
        target_ura.append("../graytest/"+str(ura[k])+"/"+str(i)+".jpg")#変更必要


im_rows = 64 #変更必要
im_cols = 64 #変更必要
im_color = 1 #変更必要
in_shape = (im_rows, im_cols, im_color)
nb_classes = 40 #変更必要
#配列の最大値　11月追加
max_score = 0.0

#12-25修正
#ラベル
LABELS=["相澤さんの表","相澤さんの裏",
       "新井さんの表","新井さんの裏",
       "江波戸の表","江波戸の裏",
       "長谷川さんの表","長谷川さんの裏",
       "樋口さんの表","樋口さんの裏",
       "石井さんの表","石井さんの裏",
       "小島さんの表","小島さんの裏",
       "中村さんの表","中村さんの裏",
       "大野さんの表","大野さんの裏",
       "坂井さんの表","坂井さんの裏",
       "関根さんの表","関根さんの裏",
       "島田さんの表","島田さんの裏",
       "清水さんの表","清水さんの裏",
       "高橋さんの表","高橋さんの裏",
       "田中ひろきさんの表","田中ひろきさんの裏",
       "田中みゆさんの表","田中みゆさんの裏",
       "田中たいきさんの表","田中たいきさんの裏",
       "徳山さんの表","徳山さんの裏",
       "渡邉さんの表","渡邉さんの裏",
       "山口さんの表","山口さんの裏"]

model = get_model(in_shape, nb_classes)
model.load_weights('./2023hdf/2023num40karnel5-64pxbatch256.hdf5')#変更必要

#最大値を見つける関数 11月追加
def findmaxscore(per):
    global max_score
    if per!=100.0:
        if max_score<=per:
            max_score=per 

def check_photo(path):
    img=cv2.imread(path)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    karnel=np.array(laplace5 , np.float64)#変更必要
    img=cv2.filter2D(img,-1,karnel)
    img=cv2.resize(img,(im_rows,im_cols))
    plt.imshow(img,cmap="gray")
    plt.show()

    x=np.asarray(img)
    x=x.reshape(-1,im_rows,im_cols,im_color)
    x=x/255
    
    #予測
    pre=model.predict([x])[0]#画像の予測
    idx=pre.argmax()
    per=(pre[idx]*100)
    if per < 99.99999 and ("omote" in path): #閾値変更箇所
        omotepercent.append(per)
    elif per < 99.99999 and ("ura" in path): #閾値変更箇所
        urapercent.append(per)
    else:
        pass
    return(idx,per)

def check_photo_str(path):
    idx,per=check_photo(path)
    if per >= 99.99999: #閾値変更箇所
        print("この写真は、",LABELS[idx],"です。")
        print("可能性は、",per,"%です。")
        passpercent.append(per)
        #配列の最大値を見つける　11月追加
        findmaxscore(per)
    else:
        print('\033[31m'+"可能性が",per,"%なので認証できません。"+'\033[31m')
        print("認証の結果は",LABELS[idx],"でした。")
        notpasspercent.append(per)
    
if __name__=='__main__':
    number=[9,19,29,39,49,59,69,79,89,99,109,119,129,139,149]
    for i in range(60):
        print("\nここからは"+str(i)+"番目の写真です")
        check_photo_str(target_omote[i])
        for l in range(15):
            if i==number[l]:
                print("================================================")
                print("テスト対象の人が変わります")
                print("================================================")
    for i in range(60):
        print("\nここからは"+str(i)+"番目の写真です")
        check_photo_str(target_ura[i])
        for l in range(15):
            if i==number[l]:
                print("================================================")
                print("テスト対象の人が変わります")
                print("================================================")
                
    print(passpercent)
    print(notpasspercent)
    print("他人受入に用いる個数"+str(len(passpercent)))
    FAR=len(passpercent)
    print("閾値未満で識別したテスト数"+str(len(notpasspercent)))
    print("表の手だけで閾値未満で識別したテスト数"+str(len(omotepercent)))
    print("裏の手だけで閾値未満で識別したテスト数"+str(len(urapercent)))
    totalpass=len(passpercent)/120 #12-26修正
    totalnotpass=len(notpasspercent)/120
    omotenotpass=len(omotepercent)/60
    omotepass=(60-len(omotepercent))/60
    uranotpass=len(urapercent)/60
    urapass=(60-len(urapercent))/60
    print("================================================")
    #未学習の手を閾値未満で識別した割合
    print("他人拒否率"+str(round(totalnotpass*100))+"%")
    #未学習の手を閾値以上で識別した割合
    print("他人受入率"+str(round(totalpass*100))+"%")
    print("表の手だけで未学習の手を閾値未満で識別した割合"+str(round(omotenotpass*100))+"%")
    print("表の手だけで未学習の手を閾値以上で識別した割合"+str(round(omotepass*100))+"%")
    print("裏の手だけで未学習の手を閾値未満で識別した割合"+str(round(uranotpass*100))+"%")
    print("裏の手だけで未学習の手を閾値以上で識別した割合"+str(round(urapass*100))+"%")
    print("閾値以上の最大値は"+str(max_score)) #11月追加
    try:
        print("閾値以上の最小値は"+str(min(passpercent))) #11月追加 
    except:
        pass